In [1]:
import h1st as h1
h1.init(MODEL_REPO_PATH = 'models')

import pandas as pd
import numpy as np
import sklearn.metrics


In [2]:
from msg_freq_event_detector import MsgFreqEventDetectorModel
m = MsgFreqEventDetectorModel()
data = m.load_data(num_files=3)

In [3]:
m.train(data)

In [4]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m.persist()

In [5]:
m.stats

,SteeringAngle,CarSpeed,YawRate,Gx,Gy
count,11084.000000,11084.000000,11084.000000,11084.000000,11084.000000
mean,34.316763,17.158607,34.314778,34.314778,34.314778
std,1.311491,2.121101,1.359257,1.359257,1.359257
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,17.000000,33.000000,33.000000,33.000000
50%,34.000000,17.000000,34.000000,34.000000,34.000000
75%,35.000000,18.000000,35.000000,35.000000,35.000000
max,40.000000,22.000000,41.000000,41.000000,41.000000


In [6]:
from graph import WindowGenerator

df = pd.read_parquet(data['attack_files'][0])

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(m)
graph.end()
results = graph.predict({"df": df})

results["event_detection_results"]

'Gx': 0,
  'Gy': 0,
  'window_start': 343.6135736239976},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 344.01357362399756},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 344.41357362399754},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 344.8135736239975},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 345.2135736239975},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 345.61357362399747},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 346.01357362399744},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 346.4135736239974},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 346.8135736239974},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 347.2135736239974},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 347.61357362399735},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 348.01357362399733},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 348.4135736239973},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 348.8135736239973},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 349.21357362399726},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 349.61357362399724},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 350.0135736239972},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 350.4135736239972},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 350.8135736239972},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 351.21357362399715},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 351.6135736239971},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 352.0135736239971},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 352.4135736239971},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 352.81357362399706},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 353.21357362399704},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 353.613573623997},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 354.013573623997},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy': 0,
  'window_start': 354.41357362399697},
 {'SteeringAngle': 0,
  'WindowInAttack': False,
  'CarSpeed': 0,
  'YawRate': 0,
  'Gx': 0,
  'Gy

In [7]:
len(results["event_detection_results"])

4124

In [8]:
att_wins1 = len([x for x in results["event_detection_results"] if x["WindowInAttack"]])

In [9]:
from gradient_boosting_msg_classifier import GradientBoostingMsgClassifierModel

In [10]:
m2 = GradientBoostingMsgClassifierModel()
data = m2.load_data(4)
prepared_data = m2.prep(data)

len train_attack_df = 661552
len test_attack_df = 579228


In [11]:
m2.train(prepared_data)

In [12]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m2.persist()

In [13]:
m2.evaluate(prepared_data)

[[554739    181]
 [  9742  14566]]
Accuracy = 0.9829


In [14]:
m2.metrics['confusion_matrix']

array([[554739,    181],
       [  9742,  14566]])

In [22]:
from graph import WindowGenerator

class NoOp(h1.Action):
    def call(self, command, inputs):
        pass

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(h1.Decision(m, decision_field="WindowInAttack"))\
     .add(yes=m2,
          no=NoOp())
graph.end()

df = pd.read_parquet(data['attack_files'][0])

results = graph.predict({"df": df})

print(sklearn.metrics.confusion_matrix(results['injection_window_results']["Label"] == "Attack", 
                                       results['injection_window_results']["MsgIsAttack"]))
print(sklearn.metrics.accuracy_score(results['injection_window_results']["Label"] == "Attack", 
                                       results['injection_window_results']["MsgIsAttack"]))

[[354987     11]
 [    70  15169]]
0.9997812212177605


In [16]:
att_wins2 = len([x for x in results["event_detection_results"] if x["WindowInAttack"]])
att_wins2

455

In [17]:
assert att_wins2 == att_wins1

In [23]:
from utils import evaluate_event_graph

TN, FP, FN, TP = evaluate_event_graph(graph, prepared_data['test_files'])

Event-level confusion matrix
[[5699    0]
 [  17  732]]
Event TPR = 0.9773, FPR = 0.0000


In [24]:
assert FP == 0